## Import the required libraries

In [1]:
import tweepy
import csv

## Twitter API credentials

In [2]:
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""

## #authorize twitter, initialize tweepy

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

## Create a list of all counties for which we need tweets. We will iterate through this list

In [4]:
counties = ["BondCountyHD", "boonecohealth", "CU_PublicHealth", "ChiPublicHealth", "CookCtyHealth", "DeKalbCoHD", "DuPageHD", 
            "KaneCoHealth", "kankakeehealth", "kendallhealth", "kchd92", "LakeCoHealth", "LaSalleCoHealth", "LCHD_IL", 
            "LCHD_Illinois", "Maconcountyhd", "MacoupinHealth", "McHenryCoHealth", "McLeanHealth", "vopnews", 
            "peoriaprepare", "polkcohealth", "RICO_HealthDept", "HenryStarkHD", "stclairhealth", "ERCSCHD", 
            "tazewellhealth", "Whiteside_CHC", "WillCoHealth", "WinnCoHealth", "WoodfordHealth"]

## Iterating through the list and generating a csv for each county

In [5]:
for county in counties:
    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    new_tweets = api.user_timeline(screen_name = county,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = county,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

        #transform the tweepy tweets into a 2D array that will populate the csv	
        outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8"), tweet.user.name, 
                      tweet.user.followers_count, tweet.user.friends_count, tweet.user.favourites_count, tweet.user.statuses_count,
                     tweet.retweet_count, tweet.favorite_count, tweet.favorited] for tweet in alltweets]

        #write the csv
        with open('%s_tweets.csv' % county, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["id","created_at","tweet", "twitter_handle","followers", 
                             "following", "profile_likes", "total_tweets", "retweets", "favorites", "is_favorited"])
            writer.writerows(outtweets)

getting tweets before 621298741216030720
...400 tweets downloaded so far
getting tweets before 517118483264311295
...600 tweets downloaded so far
getting tweets before 416654220444844031
...800 tweets downloaded so far
getting tweets before 325299092253204479
...1000 tweets downloaded so far
getting tweets before 280763428317036543
...1200 tweets downloaded so far
getting tweets before 226380556026392575
...1224 tweets downloaded so far
getting tweets before 220253819223810047
...1224 tweets downloaded so far
getting tweets before 822123848728543233
...400 tweets downloaded so far
getting tweets before 536905463544025087
...600 tweets downloaded so far
getting tweets before 3711530360
...602 tweets downloaded so far
getting tweets before 3390695937
...602 tweets downloaded so far
getting tweets before 1059894676747182079
...400 tweets downloaded so far
getting tweets before 960919522038468607
...600 tweets downloaded so far
getting tweets before 872090651688620031
...799 tweets downloa

...2600 tweets downloaded so far
getting tweets before 848908721271033856
...2800 tweets downloaded so far
getting tweets before 823916778124361730
...3000 tweets downloaded so far
getting tweets before 792077737209032705
...3195 tweets downloaded so far
getting tweets before 767866569577881599
...3224 tweets downloaded so far
getting tweets before 766273836299845631
...3224 tweets downloaded so far
getting tweets before 963045945620488191
...400 tweets downloaded so far
getting tweets before 758108399326363647
...600 tweets downloaded so far
getting tweets before 613738605161369599
...800 tweets downloaded so far
getting tweets before 503754475874439167
...977 tweets downloaded so far
getting tweets before 426055108343136255
...977 tweets downloaded so far
getting tweets before 620687450667675647
...226 tweets downloaded so far
getting tweets before 286857951317196799
...226 tweets downloaded so far
getting tweets before 1052626703300857856
...400 tweets downloaded so far
getting twee

...2594 tweets downloaded so far
getting tweets before 442005671186161665
...2715 tweets downloaded so far
getting tweets before 349890112890474499
...2715 tweets downloaded so far
getting tweets before 690170429235949568
...127 tweets downloaded so far
getting tweets before 1022264226050658303
...400 tweets downloaded so far
getting tweets before 910600390424645636
...600 tweets downloaded so far
getting tweets before 832281927579234309
...800 tweets downloaded so far
getting tweets before 766393361922007039
...1000 tweets downloaded so far
getting tweets before 686646590476578816
...1199 tweets downloaded so far
getting tweets before 608648186329223167
...1399 tweets downloaded so far
getting tweets before 534798464832786432
...1599 tweets downloaded so far
getting tweets before 460429969995599872
...1799 tweets downloaded so far
getting tweets before 375710072808411135
...1999 tweets downloaded so far
getting tweets before 265497885896998912
...2052 tweets downloaded so far
getting 